In [14]:
import networkx as nx

from regraph import Rule, plot_rule
from regraph.neo4j.hierarchy import Neo4jHierarchy
import regraph.neo4j.cypher_utils as cypher

## Initializing Neo4j database

0. When installing neo4j you will be asked to choose login/password for you dbs (here its "neo4j"/"admin"), if you choose other -- change in the cell below.
1. To start neo4j server run `sudo service neo4j start`
2. Check status by running `sudo service neo4j status`. Here you can check the _bolt_ port, change the cell below if different from 7687
3. You can query the db by using the neo4j browser, the address can be found also in the result of 'status', e.g. "Remote interface available at http://localhost:7474/".

In [30]:
# initialize the neo4j driver, wrappped into Neo4jGraph object
h = Neo4jHierarchy(uri="bolt://localhost:7687", user="neo4j", password="admin")

## Adding a graph to the hierarchy

In [33]:
h.add_graph('graphB')
b = h.access_graph('graphB')
nodes = [
    ("a", {"shape": ["circle"], "color": ["white"]}),
    ("b", {"shape": ["circle"], "color": ["black"]}),
    ("d", {"shape": ["circle"], "color": ["grey"]}),
    ("c", {"shape": ["square"], "color": ["white", "black"]})
]
edges = [
    ("a", "b"),
    ("b", "c"),
    ("b","d")
]
b.add_nodes_from(nodes)
b.add_edges_from(edges)

In [17]:
h.add_graph('graphC')
c = h.access_graph('graphC')
nodes = [
    ("a", {"shape": ["square"], "color": ["white"]}),
    ("b", {"shape": ["square"], "color": ["black"]}),
    ("c", {"shape": ["circle"], "color": ["white", "black"]}),
    ("d", {"shape": ["triangle"], "color":["grey"]})
]
edges = [
    ("a", "b"),
    ("c", "b"),
    ("c", "c"),
    ("c", "d")
]
c.add_nodes_from(nodes)
c.add_edges_from(edges)

In [18]:
h.add_graph('graphD')
d = h.access_graph('graphD')
nodes = [
    ("a", {"shape": ["circle"], "color": ["white", "black", "grey"]}),
    ("b", {"shape": ["square"], "color": ["white", "black"]}),
    ("c", {"shape": ["triangle"], "color":["grey"]})
]
edges = [
    ("a", "b"),
    ("a", "a"),
    ("b", "b"),
    ("a", "c")
]
d.add_nodes_from(nodes)
d.add_edges_from(edges)

## Typing

In [19]:
mappingBD = {
    "a":"a",
    "b":"a",
    "c":"b",
    "d":"a"
}
h.add_typing('graphB', 'graphD', mappingBD)
mappingCD = {
    "a":"b",
    "b":"b",
    "c":"a",
    "d":"c"
}
h.add_typing('graphC', 'graphD', mappingCD)

## Pullback

In [20]:
h.pullback('graphB', 'graphC', 'graphD', 'graphA')

OPTIONAL MATCH (n:graphB)-[:typing]->(:graphD)<-[:typing]-(m:graphC)

//Perform the intersection of the properties of n, m
WITH [] as new_props, m, n
WITH new_props + REDUCE(pairs = [], k in keys(n) | 
	CASE WHEN ALL(others in [m] WHERE k in keys(others))
	THEN
		pairs + REDUCE(inner_pairs = [], v in n[k] | 
			CASE WHEN ALL(others in [m] WHERE v in others[k])
			THEN
				inner_pairs + {key: k, value: v}
			ELSE
				inner_pairs
			END)
	ELSE
		pairs
	END) as new_props, m, n
WITH apoc.map.groupByMulti(new_props, 'key') as new_props, m, n
WITH apoc.map.fromValues(REDUCE(pairs=[], k in keys(new_props) | 
	pairs + [k, REDUCE(values=[], v in new_props[k] | 
		values + CASE WHEN v.value IN values THEN [] ELSE v.value END)])) as new_props, m, n
CREATE (new_node_a:node:graphA) 
SET new_node_a.id = toString(id(new_node_a)) 
WITH toString(id(new_node_a)) as id_var , new_props, m, new_node_a, n
SET new_node_a += new_props
WITH m, new_node_a, n 
MERGE (new_node_a)-[:typing {  }]->(n)

MERGE (new_n

## Pushout

In [24]:
h.remove_graph('graphD')

In [22]:
h.pushout('graphA', 'graphB', 'graphC', 'graphD')


//We copy the nodes of B in D
MATCH (n:node:graphB)
WITH [] as sucIgnore, [] as predIgnore, n 
// Match successors and out-edges of a node to be cloned in the clone graph
OPTIONAL MATCH (n)-[out_edge:edge]->(:node:graphB)-[:typing]->(suc:node:graphD)
WHERE NOT suc.id IS NULL AND NOT suc.id IN sucIgnore
WITH collect({neighbor: suc, edge: out_edge}) as suc_maps, predIgnore, n 
// match predecessors and in-edges of a node to be cloned in the clone graph
OPTIONAL MATCH (pred:node:graphD)<-[:typing]-(:node:graphB)-[in_edge:edge]->(n) 
WHERE NOT pred.id IS NULL AND NOT pred.id IN predIgnore
WITH collect({neighbor: pred, edge: in_edge}) as pred_maps, suc_maps, n 
// create a node corresponding to the clone
CREATE (m:node:graphD) 
MERGE (n)-[:typing]->(m)WITH m, toString(id(m)) as m_id_var, n.id as original_old, pred_maps, suc_maps, n 
// set the id property of the original node to NULL
SET n.id = NULL
// copy all the properties of the original node to the clone
SET m = n
// set id property o

In [25]:
h.add_graph('graphD')

In [32]:
h.clear()
h.drop_all_constraints()